In [ ]:
import json
import os
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox

# load configs
with open('../config.json') as f:
    config = json.load(f)

indicators_path = config['indicators_path'] #indicators data
graphml_folder = config['models_graphml_path']

## How many unsimplified node edges were there?

Extract from logs

In [ ]:
# this is the log from the download process
with open('/data/wc/logs/osmnx_2020-04-29.log') as f:
    lines = f.readlines()

In [ ]:
node_count = 0
edge_count = 0
for line in lines:
    if 'Created graph with' in line:
        
        n_start = line.find('with ') + 5
        n_end = line.find(' nodes')
        nodes = int(line[n_start:n_end].replace(',', ''))
        node_count += nodes

        m_start = line.find('and ') + 4
        m_end = line.find(' edges')
        edges = int(line[m_start:m_end].replace(',', ''))
        edge_count += edges

In [ ]:
print(node_count, edge_count)
# prints 150944806 301110128

## Validate networks elevations

In [ ]:
df = pd.read_csv(indicators_path)
df.shape

In [ ]:
# correlation btw mean node elevation and average UC elevation (EORC and JAXA)
df[['elev_mean', 'avg_elevation']].corr()

In [ ]:
diff = df['elev_mean'] - df['avg_elevation']
diff.describe()

## Check intersection cleaning tolerance

In [ ]:
def get_filepaths(row):
    country_folder = row['country'] + '-' + row['country_iso']
    filename = row['core_city'] + '-' + str(row['uc_id'])
    filepath = os.path.join(graphml_folder, country_folder, filename) + '.graphml'
    return filepath

In [ ]:
def save_intersections(filepath, save_graph=True):
    # load the network
    tolerance = 10
    G = ox.load_graphml(filepath)
    G_proj = ox.project_graph(G)

    # get the clean intersections
    intersections_clean = ox.clean_intersections(G_proj,
                                                 tolerance=tolerance,
                                                 dead_ends=False)
    intersections_clean.name = 'geometry'
    intersections_clean = gpd.GeoDataFrame(intersections_clean, crs=G_proj.graph['crs'])
    intersections_clean_latlng = ox.project_gdf(intersections_clean, to_latlong=True)

    # get all intersections (ie non dead-end nodes)
    node_ids = set(G.nodes())
    intersections = [node for node, count in G.graph['streets_per_node'].items() if (count > 1) and (node in node_ids)]
    gdf_nodes = ox.graph_to_gdfs(G, edges=False, node_geometry=True)
    intersections = gdf_nodes.loc[intersections]
    
    # save graph, intersections, and clean intersections GPKGs to disk
    filename = filepath[filepath.rfind('/')+1:-8]
    intersections.to_file(f'/home/geoff/Desktop/{filename}-ints.gpkg', driver='GPKG')
    intersections_clean.to_file(f'/home/geoff/Desktop/{filename}-ints-clean.gpkg', driver='GPKG')
    
    if save_graph:
        ox.save_graph_geopackage(G, filename=f'{filename}-graph.gpkg', folder='/home/geoff/Desktop')

In [ ]:
ratio = df['intersect_count_clean'] / df['intersect_count']
df['ratio'] = ratio
ratio.describe()

In [ ]:
df.sort_values('length_mean')[['country', 'core_city', 'uc_id', 'intersect_count', 'ratio', 'length_mean']].head(15)

Code below saves each of these networks, their intersections, and their clean intersections to desktop to visually inspect in QGIS. All are examples of code intersection cleaning to get better count/density indicators, except for Kismayo which does not work well because tiny footpaths in a slum are digitized as residential roads on OSM, so they are very dense.

In [ ]:
# fine grain networks
cols = ['world_region', 'country', 'core_city', 'uc_id', 'area', 'n', 'intersect_count', 'intersect_count_clean', 'ratio']
idx = ratio.sort_values().index
x = df.loc[idx, cols].head(200)
#x = df[df['world_region'].isin(['Europe', 'Northern America'])]
uc_ids = [883, 706, 1027, 2234, 10241, 5572]
x = df[df['uc_id'].isin(uc_ids)][cols]
x

In [ ]:
idx = x.index
filepaths = df.loc[idx].apply(get_filepaths, axis='columns').to_list()
for filepath in filepaths:
    save_intersections(filepath)

In [ ]:
# coarse grain networks: PHX, Buenos Aires
uc_ids = [79, 1105]
x = df[df['uc_id'].isin(uc_ids)][cols]
idx = x.index
filepaths = df.loc[idx].apply(get_filepaths, axis='columns').to_list()
for filepath in filepaths:
    save_intersections(filepath, save_graph=False)